<a href="https://colab.research.google.com/github/ykitaguchi77/PCAF_AI_Project/blob/main/Split_image_into_batches_224px.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Divide fundus photograph into batch**

2000x2000pxの画像を、224px四方のバッチに分解する

In [ ]:
# prompt: mount gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
We divided the 2000×2000 pixel images into 224×224 pixel patches with a stride of 180 pixels (19.6% overlap)...
"""
import os
import glob
from PIL import Image
import numpy as np
from pathlib import Path
import time
from datetime import datetime
from tqdm.notebook import tqdm
import math

def create_processing_log(output_dir):
    """ログファイルを作成"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_filename = os.path.join(output_dir, f"processing_log_{timestamp}.txt")
    return log_filename

def log_processing_info(log_file, message):
    """ログを記録"""
    with open(log_file, 'a', encoding='utf-8') as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"[{timestamp}] {message}\n")

def calculate_patch_info(image_size, patch_size, stride):
    """パッチ情報を計算"""
    width, height = image_size
    # 完全なグリッドのパッチ数を計算
    n_patches_w = math.floor((width - patch_size) / stride) + 1
    n_patches_h = math.floor((height - patch_size) / stride) + 1

    # 開始位置のリストを生成
    x_positions = [i * stride for i in range(n_patches_w)]
    y_positions = [i * stride for i in range(n_patches_h)]

    return {
        'n_patches_w': n_patches_w,
        'n_patches_h': n_patches_h,
        'total_patches': n_patches_w * n_patches_h,
        'x_positions': x_positions,
        'y_positions': y_positions,
        'overlap_ratio': (patch_size - stride) / patch_size * 100
    }

def process_directory(input_dir, output_dir, patch_size=224, stride=180):
    """ディレクトリ内の全PNG画像を処理"""
    start_time = time.time()

    # 出力ディレクトリの作成
    os.makedirs(output_dir, exist_ok=True)

    # ログファイルの作成
    log_file = create_processing_log(output_dir)
    log_processing_info(log_file, "=== 処理パラメータ ===")
    log_processing_info(log_file, f"パッチサイズ: {patch_size}x{patch_size} pixels")
    log_processing_info(log_file, f"ストライド: {stride} pixels")
    log_processing_info(log_file, f"オーバーラップ: {patch_size - stride} pixels")
    log_processing_info(log_file, f"オーバーラップ率: {(patch_size - stride) / patch_size * 100:.1f}%")

    # 画像パスの取得
    image_paths = glob.glob(os.path.join(input_dir, "*.png"))
    log_processing_info(log_file, f"\n処理対象画像数: {len(image_paths)}")

    processing_stats = {}

    # 画像処理のメインループ
    for image_path in tqdm(image_paths, desc="画像処理中", unit="枚"):
        original_name = Path(image_path).stem
        log_processing_info(log_file, f"\n=== 画像処理開始: {original_name} ===")

        try:
            # 画像読み込み
            img = Image.open(image_path)
            img_array = np.array(img)
            height, width = img_array.shape[:2]

            # パッチ情報の計算
            patch_info = calculate_patch_info((width, height), patch_size, stride)

            log_processing_info(log_file, f"画像サイズ: {width}x{height}")
            log_processing_info(log_file, f"予想パッチ数: {patch_info['total_patches']}")

            patch_count = 0
            with tqdm(total=patch_info['total_patches'],
                     desc=f"{original_name} パッチ生成中",
                     unit="パッチ",
                     leave=False) as pbar:

                for y in patch_info['y_positions']:
                    for x in patch_info['x_positions']:
                        # パッチを切り出し
                        patch = img_array[y:y + patch_size, x:x + patch_size]

                        # ファイル名生成と保存
                        patch_filename = f"{original_name}_x{x:04d}_y{y:04d}.png"
                        patch_img = Image.fromarray(patch)
                        patch_img.save(os.path.join(output_dir, patch_filename))

                        patch_count += 1
                        pbar.update(1)

            # 統計情報を保存
            processing_stats[original_name] = {
                'image_size': (width, height),
                'total_patches': patch_count,
                'patches_width': patch_info['n_patches_w'],
                'patches_height': patch_info['n_patches_h'],
                'overlap_pixels': patch_size - stride,
                'overlap_ratio': patch_info['overlap_ratio'],
                'status': 'success'
            }

            log_processing_info(log_file,
                f"画像処理完了: {original_name}\n"
                f"生成パッチ数: {patch_count}\n"
                f"パッチ分布: {patch_info['n_patches_w']}x{patch_info['n_patches_h']}\n"
                f"X座標範囲: 0-{max(patch_info['x_positions'])}\n"
                f"Y座標範囲: 0-{max(patch_info['y_positions'])}")

        except Exception as e:
            log_processing_info(log_file, f"エラー発生: {original_name} - {str(e)}")
            processing_stats[original_name] = {
                'status': 'error',
                'error_message': str(e)
            }

    # 処理時間の計算
    end_time = time.time()
    processing_time = end_time - start_time
    log_processing_info(log_file, f"\n=== 処理完了 ===")
    log_processing_info(log_file, f"総処理時間: {processing_time:.2f}秒")

    return processing_stats, log_file

def print_processing_summary(stats, log_file):
    """処理サマリーの表示とログ記録"""
    summary = "\n=== 処理サマリー ===\n" + "-" * 50
    total_patches = 0
    successful_images = 0
    failed_images = 0

    for image_name, info in stats.items():
        summary += f"\n\n画像: {image_name}"
        if info['status'] == 'success':
            summary += f"\n画像サイズ: {info['image_size'][0]}x{info['image_size'][1]}"
            summary += f"\nパッチ数: {info['total_patches']} ({info['patches_width']}x{info['patches_height']})"
            summary += f"\nオーバーラップ: {info['overlap_pixels']}px ({info['overlap_ratio']:.1f}%)"
            total_patches += info['total_patches']
            successful_images += 1
        else:
            summary += f"\n処理失敗: {info['error_message']}"
            failed_images += 1

    summary += f"\n\n{'-' * 50}"
    summary += f"\n総パッチ数: {total_patches}"
    summary += f"\n処理成功: {successful_images}枚"
    summary += f"\n処理失敗: {failed_images}枚"
    summary += f"\n{'-' * 50}"

    print(summary)
    log_processing_info(log_file, summary)


# メイン処理の実行
input_directory = "/content/drive/MyDrive/Deep_learning/PCAF_AI_Project/Patient27R"
output_directory = "/content/drive/MyDrive/Deep_learning/PCAF_AI_Project/Patient27R/batch_img"

# 処理パラメータ
patch_size = 224    # パッチサイズ
stride = 180        # ストライド（約19.6%オーバーラップ）

print(f"処理を開始します...")
print(f"入力ディレクトリ: {input_directory}")
print(f"出力ディレクトリ: {output_directory}")

# 処理実行
stats, log_file = process_directory(
    input_directory,
    output_directory,
    patch_size=patch_size,
    stride=stride
)

# 結果表示
print_processing_summary(stats, log_file)
print(f"\nログファイルは以下に保存されました: {log_file}")



処理を開始します...
入力ディレクトリ: /content/drive/MyDrive/Deep_learning/PCAF_AI_Project/Patient27R
出力ディレクトリ: /content/drive/MyDrive/Deep_learning/PCAF_AI_Project/Patient27R/batch_img


画像処理中:   0%|          | 0/7 [00:00<?, ?枚/s]

pt27R_1M_Photo パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]

pt27R_1M_AF パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]

pt27R_1_pre_Photo パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]

pt27R_0_pre_AF パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]

pt27R_PC_0M_Photo パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]

pt27R_1Y_Photo パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]

pt27R_1Y_AF パッチ生成中:   0%|          | 0/100 [00:00<?, ?パッチ/s]


=== 処理サマリー ===
--------------------------------------------------

画像: pt27R_1M_Photo
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

画像: pt27R_1M_AF
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

画像: pt27R_1_pre_Photo
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

画像: pt27R_0_pre_AF
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

画像: pt27R_PC_0M_Photo
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

画像: pt27R_1Y_Photo
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

画像: pt27R_1Y_AF
画像サイズ: 2000x2000
パッチ数: 100 (10x10)
オーバーラップ: 44px (19.6%)

--------------------------------------------------
総パッチ数: 700
処理成功: 7枚
処理失敗: 0枚
--------------------------------------------------

ログファイルは以下に保存されました: /content/drive/MyDrive/Deep_learning/PCAF_AI_Project/Patient27R/batch_img/processing_log_20250120_130132.txt
